# Demo: Multi-echo preprocessing in Neurodesk (fMRIPrep 25.2.3 → tedana)

This demo runs on **OpenNeuro ds005123**, **sub-10317** only.

Key choices:
- Run **fMRIPrep first**, then **tedana**
- Keep **only MNI152NLin6Asym** outputs
- Keep notebook code **robust and readable** (no `!bash -lc ...` quoting)

---


## 0) Set paths (edit PROJ if you prefer persistent storage)

If you want Neurodesk persistent storage, a common choice is something like:

`/neurodesktop-storage/neurodesktop-storage/<your-folder>`


In [ ]:
import os
from pathlib import Path

SUB = "10317"

# Change this if you prefer a persistent location:
PROJ = Path.home() / "ds005123_me_demo"

BIDS = PROJ / "bids"
LOGS = PROJ / "logs"

FMRIPREP_OUT = PROJ / "derivatives" / "fmriprep-25.2.3"
WORK = PROJ / "work"
LICENSES = PROJ / "licenses"

TEDANA_OUT = PROJ / "derivatives" / "tedana"

TEMPLATEFLOW_HOME = PROJ / "templateflow"
MPLCONFIGDIR = PROJ / "mplconfigdir"

for p in [PROJ, BIDS, LOGS, FMRIPREP_OUT, WORK, LICENSES, TEDANA_OUT, TEMPLATEFLOW_HOME, MPLCONFIGDIR]:
    p.mkdir(parents=True, exist_ok=True)

# Export variables so %%bash cells can use them
os.environ.update({
    "SUB": SUB,
    "PROJ": str(PROJ),
    "BIDS": str(BIDS),
    "LOGS": str(LOGS),
    "FMRIPREP_OUT": str(FMRIPREP_OUT),
    "WORK": str(WORK),
    "LICENSES": str(LICENSES),
    "TEDANA_OUT": str(TEDANA_OUT),
    "TEMPLATEFLOW_HOME": str(TEMPLATEFLOW_HOME),
    "MPLCONFIGDIR": str(MPLCONFIGDIR),
})

print("PROJ:", PROJ)
print("BIDS:", BIDS)
print("FMRIPREP_OUT:", FMRIPREP_OUT)
print("TEDANA_OUT:", TEDANA_OUT)


## 1) Load fMRIPrep (Neurodesk module)

Neurodesk’s example notebooks load modules inside Jupyter with the `module` helper.


In [ ]:
import module

# Load the version you specified
await module.load("fmriprep/25.2.3")
await module.list()

# Sanity check: should print the fmriprep path/version
import shutil, subprocess
print("which fmriprep:", shutil.which("fmriprep"))
subprocess.run(["fmriprep", "--version"], check=False)


## 2) DataLad: install dataset + get `sub-10317` (quiet-ish)

We log output to `logs/` and show only the last few lines.


In [ ]:
%%bash
set -euo pipefail

cd "$PROJ"

# Install (idempotent). If it already exists, this is quick.
datalad -l error install -s https://github.com/OpenNeuroDatasets/ds005123.git bids   > "$LOGS/datalad_install.log" 2>&1 || true
tail -n 12 "$LOGS/datalad_install.log" || true

# Get only this participant
cd "$BIDS"
datalad -l error get "sub-$SUB" > "$LOGS/datalad_get_sub-${SUB}.log" 2>&1
tail -n 12 "$LOGS/datalad_get_sub-${SUB}.log"

# Quick check
ls "$BIDS/sub-$SUB/func" | head


## 3) FreeSurfer license (required by fMRIPrep)

This demo expects your FreeSurfer license at `~/.license` and copies it into the project for a stable path.


In [ ]:
%%bash
set -euo pipefail

test -r "$HOME/.license" || (echo "ERROR: FreeSurfer license not found at ~/.license" && exit 1)
cp -f "$HOME/.license" "$LICENSES/fs_license.txt"
ls -l "$LICENSES/fs_license.txt"


## 4) Run fMRIPrep (MNI152NLin6Asym only; echo-wise outputs)

To avoid the FreeSurfer `subjects_dir` crash, we **create** a subjects directory and pass it explicitly with
`--fs-subjects-dir` (matching the Neurodesk example style).

Outputs (after success):
- report: `derivatives/fmriprep-25.2.3/sub-10317.html`
- echo-wise MNI preproc: `.../func/*echo-*_space-MNI152NLin6Asym_desc-preproc_bold.nii.gz`


In [ ]:
%%bash

# Follow the Neurodesk example style: set SUBJECTS_DIR, create it, and run fMRIPrep.
export SUBJECTS_DIR="$FMRIPREP_OUT/sourcedata/freesurfer"
export APPTAINERENV_SUBJECTS_DIR="$SUBJECTS_DIR"
mkdir -p "$SUBJECTS_DIR"

# Optional caches (keeps repeated downloads/config tidy)
mkdir -p "$TEMPLATEFLOW_HOME" "$MPLCONFIGDIR"

fmriprep "$BIDS" "$FMRIPREP_OUT" participant \
  --participant-label "$SUB" \
  --stop-on-first-crash \
  --skip-bids-validation \
  --me-output-echos \
  --output-spaces MNI152NLin6Asym \
  --fs-no-reconall \
  --fs-license-file "$LICENSES/fs_license.txt" \
  --fs-subjects-dir "$SUBJECTS_DIR" \
  -w "$WORK" \
  --nthreads 14 --omp-nthreads 1 --mem-mb 24000 \
  -v \
  || echo "fMRIPrep failed (exit $?)"


In [ ]:
%%bash
set -euo pipefail

# Quick checks
ls -l "$FMRIPREP_OUT/sub-$SUB.html"
ls "$FMRIPREP_OUT/sub-$SUB/func" | grep "space-MNI152NLin6Asym" | head


## 5) tedana: install (no module) + run on one MNI-space run

We run tedana on **one run** to keep this demo short.

This example targets:
- `task-doors_run-1`
- `part-mag`

If you want a different run/part, change `RUN` / `PART` below.


In [ ]:
import os, glob, json
from pathlib import Path

RUN = "task-doors_run-1"
PART = "part-mag"

# Build list of MNI echo-wise fMRIPrep outputs for this run
echo_glob = str(Path(os.environ["FMRIPREP_OUT"]) / f"sub-{SUB}/func/sub-{SUB}_{RUN}_echo-*_{PART}_space-MNI152NLin6Asym*_desc-preproc_bold.nii.gz")
ECHOS = sorted(glob.glob(echo_glob))

if not ECHOS:
    raise RuntimeError(f"No MNI echo-wise fMRIPrep outputs found for {RUN} {PART}.\nTried: {echo_glob}")

print("Echo-wise inputs:")
for p in ECHOS:
    print(" ", Path(p).name)

# Extract TEs from raw BIDS JSON sidecars
json_glob = str(Path(os.environ["BIDS"]) / f"sub-{SUB}/func/sub-{SUB}_{RUN}_echo-*_{PART}_bold.json")
J = sorted(glob.glob(json_glob))
if not J:
    raise RuntimeError(f"No JSON sidecars found. Tried: {json_glob}")

TEs = []
for jp in J:
    with open(jp) as f:
        TEs.append(str(json.load(f)["EchoTime"]))

print("TEs (seconds):", " ".join(TEs))

# Export for bash cell
os.environ["RUN"] = RUN
os.environ["PART"] = PART
os.environ["TE_LIST"] = " ".join(TEs)


In [ ]:
%%bash

# No Neurodesk module for tedana in this environment, so install it.
python3 -m pip install --user -q tedana

# Ensure the user install location is on PATH
export PATH="$HOME/.local/bin:$PATH"
which tedana
tedana --version


In [ ]:
%%bash
set -euo pipefail

export PATH="$HOME/.local/bin:$PATH"

OUTDIR="$TEDANA_OUT/sub-$SUB/func/sub-${SUB}_${RUN}_${PART}_space-MNI152NLin6Asym"
mkdir -p "$OUTDIR"

LOG="$LOGS/tedana_sub-${SUB}_${RUN}_${PART}.log"

# Collect echo-wise inputs (stable sort)
mapfile -t ECHOS < <(ls "$FMRIPREP_OUT/sub-$SUB/func/sub-${SUB}_${RUN}_echo-*_${PART}_space-MNI152NLin6Asym"*"_desc-preproc_bold.nii.gz" | sort -V)

tedana   -d "${ECHOS[@]}"   -e ${TE_LIST}   --convention bids   --out-dir "$OUTDIR"   --prefix "sub-${SUB}_${RUN}_${PART}_space-MNI152NLin6Asym"   --fittype curvefit   --overwrite   > "$LOG" 2>&1   || { echo "tedana failed. Last 120 lines:"; tail -n 120 "$LOG"; exit 1; }

echo "tedana finished. Last 30 lines:"
tail -n 30 "$LOG"

echo ""
echo "Outputs:"
ls -lh "$OUTDIR" | head
